# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770657462207                   -0.53    8.0         
  2   -2.772133372050       -2.83       -1.30    1.0    175ms
  3   -2.772169066506       -4.45       -2.67    1.0    192ms
  4   -2.772170719097       -5.78       -3.90    2.0    193ms
  5   -2.772170722884       -8.42       -4.45    2.0    207ms
  6   -2.772170723008       -9.91       -5.30    1.0    205ms
  7   -2.772170723015      -11.17       -5.94    2.0    204ms
  8   -2.772170723015      -12.97       -6.54    1.0    215ms
  9   -2.772170723015      -14.05       -7.66    2.0    229ms
 10   -2.772170723015      -14.35       -7.92    2.0    246ms
 11   -2.772170723015   +    -Inf       -9.07    1.0    185ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770795739561                   -0.52    9.0         
  2   -2.7

1.7735579981995528

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770721209780                   -0.53    8.0         
  2   -2.772050161346       -2.88       -1.31    1.0    162ms
  3   -2.772082555796       -4.49       -2.60    1.0    222ms
  4   -2.772083412630       -6.07       -3.84    2.0    206ms
  5   -2.772083417213       -8.34       -4.14    2.0    209ms
  6   -2.772083417799       -9.23       -5.48    1.0    235ms
  7   -2.772083417810      -10.93       -5.96    2.0    192ms
  8   -2.772083417811      -12.98       -6.77    1.0    174ms
  9   -2.772083417811      -14.88       -7.56    2.0    242ms
 10   -2.772083417811      -13.95       -7.69    2.0    225ms
 11   -2.772083417811      -14.24       -9.06    1.0    216ms

Polarizability via ForwardDiff:       1.7725349718014076
Polarizability via finite difference: 1.7735579981995528
